In [1]:
include("fid_utils.jl");

# Calibration:
First calculate how much population error is introduced by rotation angle error when driving a $\pi$ pulse. We'll use this to convert $\pi$ pulse population error into variance of rotation angle.

In [2]:
function fid_π(εs)
    sw = SysWrapper(sys2, εs)
    ψ = R2_ratio(sw, 1, 2, 1) * sys2.ψ2s[1]
    return norm(sys2.proj2s[2, 2] * ψ)^2
end
display(fid_π([0, 0]))
m_π = ForwardDiff.hessian(fid_π, [0.0, 0.0])
display(m_π)
calib_scale = m_π[1, 1];

1.0

2×2 Matrix{Float64}:
 -4.9348        3.70051e-32
  3.70051e-32  -4.9348

## $\pi$ pulse calibration
The $\pi$ pulse population infidelity for the two chambers are measured to be $0.5%$ and $1%$.

In [3]:
err_a::Float64 = 0.5e-2;
err_b::Float64 = 0.5e-2;

In [4]:
function disp_sys(sys, cb, name)
    @assert all(cb(sys, [0, 0]) .≈ (1, 1))
    mp = ForwardDiff.hessian((x)->cb(sys, x)[1], [0.0, 0.0])
    mc = ForwardDiff.hessian((x)->cb(sys, x)[2], [0.0, 0.0])
    err_p = mp[1, 1] * (err_a + err_b) / calib_scale * 100
    err_c = mc[1, 1] * (err_a + err_b) / calib_scale * 100
    display(Markdown.parse("## Error for $(name)"))
    display(Markdown.parse("**Population: $(round(err_p, digits=2))%**"))
    display(Markdown.parse("**Parity: $(round(err_c, digits=2))%**"))
    display(Markdown.parse("**Total: $(round((err_p + err_c) / 2, digits=2))%**"))
    display(mp)
    display(mc)
    display((mp .+ mc) ./ 2)
    return err_p, err_c, (err_p + err_c) / 2
end

disp_sys (generic function with 1 method)

In [5]:
struct EmitWrapper{Sys,Ta,Tb,P}
    sys::Sys
    pa::Ta
    pb::Tb
    projs::Vector{P}
end
function EmitWrapper(sys, pa, pb)
    proja1 = sys.proj1s[1] * sqrt(pa)
    proja0 = sys.proj1s[1] * sqrt(1 - pa) + sys.proj1comps[1]
    projb1 = sys.proj1s[1] * sqrt(pb)
    projb0 = sys.proj1s[1] * sqrt(1 - pb) + sys.proj1comps[1]
    projs = [proja0 ⊗ projb0, proja1 ⊗ projb0 + proja0 ⊗ projb1, proja1 ⊗ projb1]
    return EmitWrapper(sys, pa, pb, projs)
end
function detect_success(ρ, ew::EmitWrapper)
    return 2 * apply(ρ, ew.projs[3]) + apply(ρ, ew.projs[2])
end
function detect_fail(ρ, ew::EmitWrapper)
    return apply(ρ, ew.projs[3]) + apply(ρ, ew.projs[2]) + apply(ρ, ew.projs[1])
end

detect_fail (generic function with 1 method)

In [6]:
pa::Float64 = 0.8;
pb::Float64 = 0.8;

In [7]:
function init_state(sw::SysWrapper{Sys} where Sys<:System{2}, ew::EmitWrapper, success)
    ψ = R2_ratio(sw, 1, 2, 1 / 2) * sw.sys.ψ2s[1]
    ρ = projector(ψ)
    ρ = 1 in success ? detect_success(ρ, ew) : detect_fail(ρ, ew)
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1))
    return 2 in success ? detect_success(ρ, ew) : detect_fail(ρ, ew)
end

function fid01_10(sys::System{2}, εs)
    sw = SysWrapper(sys, εs)
    ew = EmitWrapper(sys, pa, pb)
    ρ = init_state(sw, ew, (1, 2))
    # 01 -> 10
    ρ = apply(ρ, sys.proj2sub2[1, 2])
    normalize!(ρ)
    P_odd = real(tr(apply(ρ, sys.proj2s[1, 2])) + tr(apply(ρ, sys.proj2s[2, 1])))
    ρ = apply(ρ, R2_ratio(sw, 1, 2, 1 / 2))
    c = real(tr(apply(ρ, sys.proj2s[1, 1])) + tr(apply(ρ, sys.proj2s[2, 2])))
    return P_odd, c
end

fid01_10 (generic function with 1 method)

# $d=2$

In [8]:
display(md"# Excitation probability **\$p=0.8\$**")
pa::Float64 = 0.8;
pb::Float64 = 0.8;
d2_70 = (f01_10=disp_sys(sys2, fid01_10, "01 -> 10"),);
display(md"# Excitation probability **\$p=1.0\$**")
pa::Float64 = 1;
pb::Float64 = 1;
d2_100 = (f01_10=disp_sys(sys2, fid01_10, "01 -> 10"),);

# Excitation probability **$p=0.8$**


## Error for 01 -> 10


**Population: 1.2%**


**Parity: 2.59%**


**Total: 1.9%**


...

**Parity: 1.5%**


**Total: 1.25%**


2×2 Matrix{Float64}:
 -4.9348  -4.9348
 -4.9348  -4.9348

2×2 Matrix{Float64}:
 -7.4022       -1.33227e-15
 -1.77636e-15  -7.4022

2×2 Matrix{Float64}:
 -6.1685  -2.4674
 -2.4674  -6.1685